In [ ]:
!pip install huggingface_hub transformers bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U transformers accelerate bitsandbytes torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 모델 불러오기

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import IA3Config, get_peft_model, prepare_model_for_kbit_training

# 4비트 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# IA3 설정
ia3_config = IA3Config(
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    feedforward_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 모델명
model_name = 'naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B'

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

# IA3를 모델에 적용
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, ia3_config)

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

### SFT 데이터

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/LLM/planty/data/sft_dataset

/content/drive/MyDrive/LLM/planty/data/sft_dataset


### 데이터 불러오기

In [ ]:
import json
from datasets import Dataset

#json 데이터 로드
with open('sft_dataset_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

dataset = Dataset.from_dict({"question": [item["question"] for item in data], "answer": [item["answer"] for item in data]})

In [ ]:
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 1242
})

In [ ]:
dataset['question'][:10]

['분갈이는 얼마에 한 번씩 하나요?',
 '분갈이에 가장 좋은 시기는?',
 '분갈이가 필요한 때를 어떻게 알 수 있나요?',
 '분갈이에 필요한 도구와 용기의 선택: 도구',
 '분갈이에 필요한 도구와 용기의 선택: 용기',
 '분갈이에 필요한 도구와 용기의 선택: 용기와 식물의 비율',
 '분갈이에 필요한 도구와 용기의 선택: 용기(화분)의 선택',
 '분갈이에 어떤 흙을 사용해야 하는가?',
 '분갈이할 때 산이나 주변에서 볼 수 있는 흙도 가능한가? 아니면, 꼭 구입한 토양만 써야 하는가?',
 '분갈이 시 흙의 양은?']

In [ ]:
dataset['answer'][:5]

['성장이 왕성하여 1년마다 사는 집이 비좁아지는 식물은 매년 봄에, 2년 정도 되어야새집이 필요한 식물은 2년마다, 성장이 아주 느린 식물은 필요한 시기에 해 준다.',
 '식물도 동물처럼 휴식이 필요하며 대부분의 식물은 한겨울을 휴식기로 선택한다. 그러나 봄이 오면 다시 새로운 생명 활동을 시작하므로 아직 꽃망울이 맺히기 전에 분갈이가 필요한 식물들은 분갈이를 한다. 그러나 너무 이른 봄보다는 좀 더 따뜻해졌을때(4~5월경) 새 집으로 이사하고 싶어 하는 식물들도 많다.',
 '분갈이가 필요할 때 식물들은 열심히 신호를 보낸다. 뿌리가 배수공을 빠져나와 소리를 지르는 광경은 흔히 볼 수 있는 풍경이다. 또한 물을 줄 때마다 이미 너무 커 버려 작은 화분이 자꾸 넘어지는 모습에서 우리는 식물의신호를 읽을 수 있다.',
 '새롭게 옮겨 심을 화분, 모종삽, 막대기, 물뿌리개, 배양토, 화분망 등',
 '식물에게 용기의 선택은 사람에게 살 집을 선택하는 것과 같다. 우리는 미관상 보기에만 좋은 집이 살기 편하다고 이야기하지는 않는다. 용기는 화분 안에 들어갈 흙의 온도와 수분의 유지, 조절에도 관여하며 식물을 더욱 아름답게, 편안하게 만드는 역할도 한다.']

### 토크나이징

In [ ]:
def tokenize_function(examples):
    # 프롬프트 생성
    prompt = [f"<|system|>\n식물에 관한 질문에 정확하게 답변해주세요.</|system|>\n<|user|>\n{q}</|user|>\n<|assistant|>\n{a}</|assistant|>"
             for q, a in zip(examples["question"], examples["answer"])]

    # 토크나이징 (truncation과 padding 적용)
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # labels 생성 (padding 부분은 -100 처리)
    tokenized["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in ids]
        for ids in tokenized["input_ids"]
    ]

    return tokenized

# 데이터셋을 토크나이징 적용하여 변환
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1242
})

## 파인튜닝

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir = "./HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=False,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,
)

# 데이터셋 분할
train_val_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_data = train_val_split["train"]
val_data = train_val_split["test"]

# Trainer 정의 (data_collator 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator
)

# 모델 학습 시작
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mailharu365 (mailharu365-dankook-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,3.654186
2,No log,3.595820
3,No log,3.585725


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=105, training_loss=3.5290887741815475, metrics={'train_runtime': 1072.551, 'train_samples_per_second': 3.124, 'train_steps_per_second': 0.098, 'total_flos': 1.3992504204460032e+16, 'train_loss': 3.5290887741815475, 'epoch': 3.0})

## 모델 저장

In [ ]:
model.save_pretrained('./HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3')
tokenizer.save_pretrained("./HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3")

('./HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/tokenizer_config.json',
 './HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/special_tokens_map.json',
 './HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/vocab.json',
 './HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/merges.txt',
 './HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/added_tokens.json',
 './HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/tokenizer.json')

In [ ]:
from huggingface_hub import login, create_repo
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Hugging Face 로그인
login(token='your_api_token')

# 2. 저장소 생성 (저장소가 없다면 생성)
repo_id = "yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3"  # 원하는 repo_id 입력
create_repo(repo_id, private=False)  # 공개 저장소 생성 (비공개는 private=True)

# 3. 모델과 토크나이저 로드 (로컬 경로)
model_path = './HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3'  # 로컬에 저장된 모델 경로
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 4. 모델과 토크나이저를 Hugging Face Hub에 업로드
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

adapter_model.safetensors:   0%|          | 0.00/798k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/commit/e7699e779bde7e7f7dd9f6a209e78efa2d09c3a8', commit_message='Upload tokenizer', commit_description='', oid='e7699e779bde7e7f7dd9f6a209e78efa2d09c3a8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3', endpoint='https://huggingface.co', repo_type='model', repo_id='yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3'), pr_revision=None, pr_num=None)

### 저장한 모델 불러오기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

adapter_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

In [ ]:
# 입력 프롬프트 설정
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"

# 입력을 토큰화
inputs = tokenizer(prompt, return_tensors="pt")

# 모델을 사용하여 텍스트 생성
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
plant = tokenizer.decode(output[0], skip_special_tokens=True)

print("번역:")
print(plant)

번역:
다육식물은 겨울철에 어떻게 관리해야 하나요? 다육식물은 겨울철에 휴면기를 가져야 하며, 4주 정도 전에 생장 억제를 시작하는 것이 좋습니다. 겨울 동안에는 물을 적게 주고, 햇빛을 적게 받으며, 서늘한 온도에서 보관하는 것이 중요합니다. 겨울이 지나면 꽃을 피우기 위해 준비시키기 위해 서서히 생장 억제를 종료하고, 봄이 오면 물을 충분히 주고, 햇빛을 충분히 받으며, 적절한 온도와 환경에서 정상적으로 성장할 수 있도록 해야 합니다.
assistant
다육식물은 겨울에 휴면기를 가지는 것이 일반적인 관리 방법입니다. 이를 적절하게 관리하기 위해 다음과 같은 조치들을 고려할 수 있습니다.

1. **물주기 조절**: 겨울철에는 물을 너무 많이 주지 않습니다. 다육식물의 물 필요성이 줄어들기 때문에 겉흙이 말랐을 때 약간의 물을 주는


In [ ]:
import torch
import time

device = torch.device("cpu")

# 추론 최적화
def generate_answer(prompt, max_length=50):
    # 시간 측정 시작
    start_time = time.time()

    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 추론 시 그라디언트 계산 비활성화 (추론 성능 최적화)
    with torch.no_grad():
        # 텍스트 생성
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,  # 생성의 다양성 제어
            top_p=0.9,        # nucleus sampling
            top_k=50,         # 상위 k개 토큰 샘플링
            do_sample=True    # 샘플링을 통해 더 자연스러운 답변 생성
        )

    # 출력 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 시간 측정 종료
    end_time = time.time()

    # 소요된 시간 출력
    print(f"답변 생성 시간: {end_time - start_time:.4f} 초")

    return generated_text

# 예시 사용
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"
answer = generate_answer(prompt)
print("생성된 답변:", answer)


답변 생성 시간: 5.7944 초
생성된 답변: 다육식물은 겨울철에 어떻게 관리해야 하나요?
assistant
다육식물은 겨울철에 특별한 관리와 주의가 필요합니다. 다음은 다육식물을 건강하게 유지하기 위한 몇 가지 팁입니다.

### 


In [ ]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# 입력 프롬프트 설정
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"

# 시간 측정 시작
start_time = time.time()

# 입력을 토큰화
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 모델을 사용하여 텍스트 생성
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
plant = tokenizer.decode(output[0], skip_special_tokens=True)

# 시간 측정 종료
end_time = time.time()

# 소요된 시간 출력
print(f"답변 생성 시간: {end_time - start_time:.4f} 초")

# 생성된 답변 출력
print("번역:")
print(plant)

답변 생성 시간: 28.8858 초
번역:
다육식물은 겨울철에 어떻게 관리해야 하나요? 겨울철에 적절한 관리 방법은 다음과 같습니다.
  1. 물 주기: 다육식물의 겨울철 물 주기는 4개월 정도입니다. 겨울철 습도가 낮아 무의 성장을 촉진하는데 도움이 되므로 물 주기에 신경 써야 합니다.
  2. 광량 조절: 겨울철 광량 조절도 중요한 관리 방법 중 하나입니다. 실내 온도를 적절히 유지하여 빛이 부족하지 않도록 합니다.
  3. 온도 유지: 겨울철 온도를 14도 이상으로 유지하여 식물들이 동해를 입지 않도록 해야 합니다.

위 내용을 바탕으로 코드를 작성해보겠습니다.

```python
class Plant:
    def __init__(self, name, water_needed, light_needed, temperature, humidity):
        self.name = name
        self article_content = ""
        self.water_needed = water_needed
        self.light_needed = light_needed
        self.temperature =
